In [1]:
import tkinter as tk
from tkinter import ttk
import numpy as np
import statsmodels.stats.power as smp
import statsmodels.api as sm

def simulate_data(n_samples, n_groups, effect_size):
    # Simulate data based on effect size and number of groups
    np.random.seed(42)
    groups = np.random.randint(0, n_groups, size=n_samples)
    data = np.random.normal(loc=groups * effect_size, scale=1.0, size=n_samples)
    return groups, data

def calculate_sample_size(effect_size, num_groups, alpha, power, n_samples):
    # Simulate data
    groups, data = simulate_data(n_samples, num_groups, effect_size)

    # Calculate sample size for ANOVA
    nobs = smp.FTestAnovaPower().solve_power(
        effect_size=effect_size, k_groups=num_groups, alpha=alpha, power=power
    )

    return round(nobs)

def calculate_achieved_power(effect_size, num_groups, alpha, data):
    # Fit an OLS model to the simulated data
    groups = np.arange(num_groups)
    X = np.zeros((len(data), num_groups))
    for i in range(num_groups):
        X[data == i, i] = 1

    model = sm.OLS(data, X)
    results = model.fit()

    # Perform ANOVA on the OLS results
    anova_table = sm.stats.anova_lm(results, typ=2)

    # Extract sum of squares and degrees of freedom
    sum_sq = anova_table['sum_sq'].iloc[0]
    df = anova_table['df'].iloc[0]

    # Calculate mean square
    mse = sum_sq / df

    # Calculate observed effect size
    observed_effect_size = effect_size / np.sqrt(mse / num_groups)

    # Calculate achieved power
    achieved_power = smp.FTestAnovaPower().solve_power(
        effect_size=observed_effect_size, k_groups=num_groups, alpha=alpha, nobs=df
    )

    return observed_effect_size, achieved_power

def calculate_button_clicked():
    try:
        effect_size = float(effect_size_entry.get())
        num_groups = int(num_groups_entry.get())
        confidence_level = confidence_combobox.get()
        power = float(power_entry.get())
        n_samples = int(n_samples_entry.get())

        # Calculate sample size
        min_sample_size = calculate_sample_size(effect_size, num_groups, alpha_levels[confidence_level], power, n_samples)

        # Simulate data for achieved power calculation
        groups, data = simulate_data(n_samples, num_groups, effect_size)

        # Calculate achieved power
        observed_effect_size, achieved_power = calculate_achieved_power(effect_size, num_groups, alpha_levels[confidence_level], data)

        # Display results
        result_label.config(text=f"Minimum Sample Size Needed: {min_sample_size}\n"
                                 f"Observed Effect Size: {observed_effect_size:.4f}\n"
                                 f"Achieved Power: {achieved_power:.4f}")

    except ValueError as e:
        result_label.config(text=f"Error: {e}")
    except KeyError as e:
        result_label.config(text=f"KeyError: {e}")

# Setup GUI
root = tk.Tk()
root.title("Sample Size and Achieved Power Calculator for ANOVA")

# Define confidence levels
alpha_levels = {
    "80%": 0.2,
    "85%": 0.15,
    "90%": 0.1,
    "95%": 0.05,
    "99%": 0.01
}

# GUI components
ttk.Label(root, text="Effect Size:").pack()
effect_size_entry = ttk.Entry(root)
effect_size_entry.pack()
effect_size_entry.insert(0, "0.5")  

ttk.Label(root, text="Number of Groups:").pack()
num_groups_entry = ttk.Entry(root)
num_groups_entry.pack()
num_groups_entry.insert(0, "3")

ttk.Label(root, text="Confidence Level:").pack()
confidence_combobox = ttk.Combobox(root, values=["80%", "85%", "90%", "95%", "99%"])
confidence_combobox.pack()
confidence_combobox.set("95%")  

ttk.Label(root, text="Desired Statistical Power:").pack()
power_entry = ttk.Entry(root)
power_entry.pack()
power_entry.insert(0, "0.8")

ttk.Label(root, text="Number of Simulated Samples:").pack()
n_samples_entry = ttk.Entry(root)
n_samples_entry.pack()
n_samples_entry.insert(0, "100")

calculate_button = ttk.Button(root, text="Calculate Sample Size and Achieved Power", command=calculate_button_clicked)
calculate_button.pack()

result_label = ttk.Label(root, text="")
result_label.pack()

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ChrisSaid\anaconda3\envs\BiQ_env_20231205\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\ChrisSaid\AppData\Local\Temp\ipykernel_17984\3700287883.py", line 70, in calculate_button_clicked
    observed_effect_size, achieved_power = calculate_achieved_power(effect_size, num_groups, alpha_levels[confidence_level], data)
  File "C:\Users\ChrisSaid\AppData\Local\Temp\ipykernel_17984\3700287883.py", line 36, in calculate_achieved_power
    anova_table = sm.stats.anova_lm(results, typ=2)
  File "c:\Users\ChrisSaid\anaconda3\envs\BiQ_env_20231205\lib\site-packages\statsmodels\stats\anova.py", line 349, in anova_lm
    return anova_single(model, **kwargs)
  File "c:\Users\ChrisSaid\anaconda3\envs\BiQ_env_20231205\lib\site-packages\statsmodels\stats\anova.py", line 66, in anova_single
    design_info = model.model.data.design_info
AttributeError: 'ModelData' object has 